In [5]:
import time
import firebase_admin
from firebase_admin import credentials, db, storage
from datetime import datetime
import os
import keyboard

In [3]:
cred = credentials.Certificate("keys/touri-65f07-firebase-adminsdk-wuv71-b245c875f8.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://touri-65f07-default-rtdb.firebaseio.com/',
    'storageBucket' : 'touri-65f07.appspot.com' 
    })

# MESSAGING

In [6]:
def sendMsgToUser(msg):
    try : 
        cred = credentials.Certificate("keys/touri-65f07-firebase-adminsdk-wuv71-b245c875f8.json")
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'https://touri-65f07-default-rtdb.firebaseio.com/',
            'storageBucket' : 'touri-65f07.appspot.com' 
        })
    except:
        pass  
    
    db.reference("state").update({
        "botMsg" : msg
    })


In [4]:
def resttingState():
    try: 
        cred = credentials.Certificate("keys/touri-65f07-firebase-adminsdk-wuv71-b245c875f8.json")
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'https://touri-65f07-default-rtdb.firebaseio.com/',
            'storageBucket' : 'touri-65f07.appspot.com' 
        })
    except:
        pass  

    db.reference("autoSkills/pickPlace/").update({
        "imgRequested" : False
    })

    db.reference("state").update({
        "runningAutoCommand" : False
    })


In [10]:
resttingState()

# NAVIAGTION


In [19]:
navStackLaunchCmd =  "roslaunch stretch_navigation navigation.launch map_yaml:=${HELLO_FLEET_PATH}/maps/ai_F.yaml"
goToGymLocCmd =  "rostopic pub /move_base_simple/goal geometry_msgs/PoseStamped '{header: {stamp: now, frame_id: 'map'}, pose: {position: {x: 37.84, y: 23.03, z: 0.0}, orientation: {z: -0.37, w: -0.92}}}'"
goToAIMakerSpaceCmd = "rostopic pub /move_base_simple/goal geometry_msgs/PoseStamped '{header: {stamp: now, frame_id: 'map'}, pose: {position: {x: 17.9, y: 19.5,z: 0.0}, orientation: {z: -0.92, w: 0.39}}}'"

startPosAiMakerSpace = '''rostopic pub /initialpose geometry_msgs/PoseWithCovarianceStamped "header:
  seq: 0
  stamp:
    secs: 0
    nsecs: 0
  frame_id: ''
pose:
  pose:
    position: {x: 37.84, y: 23.03, z: 0.0}
    orientation: {x: 0.0, y: 0.0, z: -0.89, w: 0.45}
  covariance: [0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06853891945200942]'''


proximityFile = "python3 /home/hello-robot/trial/catkin_ws/src/stretch_ros/touri_ros/src/touri_perception/scripts/obstacle_alert.py"
posUpdateFile = "python3 /home/hello-robot/trial/catkin_ws/src/stretch_ros/touri_ros/src/touri_perception/scripts/pos_update.py"


In [27]:
a = "hello"
a = a.upper()
a

'HELLO'

### UPDATE MAP COORDS

In [126]:
def updateBotPosInMap(x,y):
    '''
    xScale = 16.2 / (700-160) 
    yScale = 18.7 / (1050 - 500)

    print(f"X: {xScale}     Y: {yScale}")
    '''

    try : 
        cred = credentials.Certificate("keys/touri-65f07-firebase-adminsdk-wuv71-b245c875f8.json")
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'https://touri-65f07-default-rtdb.firebaseio.com/',
            'storageBucket' : 'touri-65f07.appspot.com' 
        })
    except:
        pass  

    scale = 0.035

    x = x * scale
    y = y * scale
    x = x + 0.12
    y =  0.915 - y

    assert abs(x) <= 1, "X must be between 0 and 1"
    assert abs(y) <= 1, "Y must be between 0 and 1"
    db.reference("autoSkills/navigation/botPos").update({
        "x" : x,
        "y": y
    })

In [29]:
x,y = 0,4
test = 'rosservice call /tap_service "x_tap: ' + str(x) + ' y_tap: ' + str(y) + '"'
print(test)

rosservice call /tap_service "x_tap: 0 y_tap: 4"


### PROXIMITY

In [8]:
def updateProximity(top, back, left, right):

    assert top in ('ok','close','too close'), "Front proximity can only take values of ('ok','close','too close')"
    assert back in ('ok','close','too close'), "Back proximity can only take values of ('ok','close','too close')"
    assert left in ('ok','close','too close'), "Left proximity can only take values of ('ok','close','too close')"
    assert right in ('ok','close','too close'), "Right proximity can only take values of ('ok','close','too close')"


    try : 
        cred = credentials.Certificate("keys/touri-65f07-firebase-adminsdk-wuv71-b245c875f8.json")
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'https://touri-65f07-default-rtdb.firebaseio.com/',
            'storageBucket' : 'touri-65f07.appspot.com' 
        })
    except:
        pass   

    msgEncode = {
            'ok' : 0,
            'close' : 1,
            'too close' : 2,
            }
            
    db.reference("state/navObjProximity").update({
        "top" : msgEncode[top],
        "back": msgEncode[back],
        "left": msgEncode[left],
        "right": msgEncode[right],
    })



### SAVE POSITION

In [6]:
def savePosition(x, y):
    try : 
        cred = credentials.Certificate("keys/touri-65f07-firebase-adminsdk-wuv71-b245c875f8.json")
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'https://touri-65f07-default-rtdb.firebaseio.com/',
            'storageBucket' : 'touri-65f07.appspot.com' 
        })
    except:
        pass    
            
    db.reference("state/location").update({
        'x' : x,
        'y' : y
    })


### GET LAST SAVED VALUE

In [ ]:
def getLastSavedPosition():
    try : 
        cred = credentials.Certificate("keys/touri-65f07-firebase-adminsdk-wuv71-b245c875f8.json")
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'https://touri-65f07-default-rtdb.firebaseio.com/',
            'storageBucket' : 'touri-65f07.appspot.com' 
        })
    except:
        pass  

    x,y = db.reference("state/location").get().values()
    return x, y



# PICK PLACE

### UPLOAD IMG

In [ ]:
def uploadImg(imgPath):
    try : 
        cred = credentials.Certificate("keys/touri-65f07-firebase-adminsdk-wuv71-b245c875f8.json")
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'https://touri-65f07-default-rtdb.firebaseio.com/',
            'storageBucket' : 'touri-65f07.appspot.com' 
        })
    except:
        pass  
    bucket = storage.bucket()
    blob = bucket.blob(imgPath)
    blob.upload_from_filename(imgPath)
    blob.make_public()
    
    db.reference("autoSkills/pickPlace").update({
        "imgSrc" : blob.public_url
    })


In [ ]:
uploadImg("pickPlaceImg.jpeg")